# Introduction

This notebook answers the following question:  

- What is the probability of a non-volunteer becoming a volunteer ?


by Fred Etter - December, 2019

In [1]:
# Import modules

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn import linear_model
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
import sklearn
from sklearn.feature_selection import SelectFromModel
from datetime import datetime
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import seaborn as sns

In [2]:
# Read in the data
df = pd.read_csv('file1_12_3.csv', low_memory=False)

In [3]:
# show the first 5 lines of the data
df.head()

,ContactId,Year,PledgeTotal,VolType,VolunteerActivityCnt,BirthYear,Gender
0,874ddbce-11cd-e111-941f-00259073dc22,2007,52.00,NaN,NaN,1977.0,female
1,9a4ddbce-11cd-e111-941f-00259073dc22,2007,80.00,NaN,NaN,1968.0,female
2,5e4edbce-11cd-e111-941f-00259073dc22,2007,10000.00,NaN,NaN,1958.0,male
3,c150dbce-11cd-e111-941f-00259073dc22,2007,120.00,NaN,NaN,NaN,male
4,0b53dbce-11cd-e111-941f-00259073dc22,2007,500.16,NaN,NaN,1947.0,female


In [4]:
# show the number of rows and columns of the original data
df.shape

(385722, 7)

In [5]:
# drop Council, Committe, Board members
df = df.drop(df[df.VolType == 'Council, Committe or Board'].index).fillna(0)

In [6]:
df = df.fillna(0)

In [7]:
df.sample(10)

,ContactId,Year,PledgeTotal,VolType,VolunteerActivityCnt,BirthYear,Gender
233549,d846b1d1-11cd-e111-941f-00259073dc22,2013,104.0,0,0.0,0.0,female
135350,beda8c60-3a92-e211-a0e0-4040184c1c1a,2012,130.0,0,0.0,0.0,female
212221,e5c799cf-11cd-e111-941f-00259073dc22,2010,24.0,0,0.0,0.0,female
249809,63ba55d0-11cd-e111-941f-00259073dc22,2015,52.0,0,0.0,1979.0,female
71833,be757cf1-7591-e611-97d4-26d4160798d6,2016,25.0,0,0.0,0.0,male
54621,4ace55d0-11cd-e111-941f-00259073dc22,2014,52.0,0,0.0,0.0,female
231015,e64cb1d1-11cd-e111-941f-00259073dc22,2012,26.0,0,0.0,0.0,female
367801,44bf55d0-11cd-e111-941f-00259073dc22,2018,1040.0,0,0.0,0.0,female
122641,11be55d0-11cd-e111-941f-00259073dc22,2011,24.0,0,0.0,0.0,female
313215,684ae8cf-11cd-e111-941f-00259073dc22,2011,1001.0,0,0.0,1972.0,male


In [8]:
df_non_only = df.drop(df[df.VolType == 'Standard'].index).fillna(0)

In [9]:
# create a series that captures the earliest year that a non-volunteer donated
a = df_non_only.groupby('ContactId')['Year'].transform('min')

In [10]:
# create a new column and populate it:
df_non_only['min_year'] = a

In [11]:
# display a random sample of 10 rows of the dataframe
df_non_only.sample(10)

,ContactId,Year,PledgeTotal,VolType,VolunteerActivityCnt,BirthYear,Gender,min_year
200842,4bb362d1-11cd-e111-941f-00259073dc22,2008,156.00,0,0.0,0.0,female,2008
32553,97c462d1-11cd-e111-941f-00259073dc22,2011,48.00,0,0.0,1965.0,female,2009
110120,04d555d0-11cd-e111-941f-00259073dc22,2009,115.68,0,0.0,1952.0,female,2007
140898,bf956034-a12b-e311-a76b-4040184c1c1a,2013,25.00,0,0.0,0.0,male,2013
316420,7c3edbce-11cd-e111-941f-00259073dc22,2012,52.00,0,0.0,0.0,female,2007
15173,52ce55d0-11cd-e111-941f-00259073dc22,2009,78.00,0,0.0,0.0,male,2007
192893,1f6bb7d0-11cd-e111-941f-00259073dc22,2007,52.00,0,0.0,0.0,male,2007
364870,5d64a383-82a7-e711-84bd-005056975e92,2017,15.00,0,0.0,0.0,male,2017
359115,380fce10-7891-e611-97d4-26d4160798d6,2016,130.00,0,0.0,0.0,female,2016
13957,21b855d0-11cd-e111-941f-00259073dc22,2009,214.24,0,0.0,1946.0,female,2007


In [12]:
# drop rows where Year = min_year
df_non_only = df_non_only.drop(df_non_only[df_non_only.Year != df_non_only.min_year].index)

In [13]:
df_non_only.sample(10)

,ContactId,Year,PledgeTotal,VolType,VolunteerActivityCnt,BirthYear,Gender,min_year
140180,4222e471-b113-4d5c-92b4-15f07f6219ff,2013,240.00,0,0.0,0.0,male,2013
162122,56e203b8-d09a-e511-908e-26d4160798d6,2015,25.00,0,0.0,0.0,female,2015
292756,8a9103d1-11cd-e111-941f-00259073dc22,2008,30.00,0,0.0,1956.0,female,2008
37458,063c15d2-11cd-e111-941f-00259073dc22,2011,5.00,0,0.0,0.0,male,2011
35367,250ec1f7-1ef9-e111-9497-001f290a401c,2012,50.00,0,0.0,0.0,male,2012
235984,f8696ebf-5bc1-419a-a246-6ef25bea814a,2013,24.00,0,0.0,0.0,male,2013
287171,48e674ce-11cd-e111-941f-00259073dc22,2008,180.36,0,0.0,1966.0,male,2008
306586,823de8cf-11cd-e111-941f-00259073dc22,2010,52.00,0,0.0,0.0,male,2010
345241,fb5bdbce-11cd-e111-941f-00259073dc22,2016,0.00,0,0.0,0.0,female,2016
315231,263215d2-11cd-e111-941f-00259073dc22,2011,104.00,0,0.0,0.0,male,2011


In [14]:
df_non_only.shape

(115554, 8)

In [15]:
df_std_only = df.drop(df[df.VolType == 0].index).fillna(0)

In [16]:
df_std_only.shape

(8439, 7)

In [17]:
df_std_only = df_std_only.drop_duplicates('ContactId')

In [18]:
df_std_only.shape

(6147, 7)

In [19]:
df_std_only.sample(5)

,ContactId,Year,PledgeTotal,VolType,VolunteerActivityCnt,BirthYear,Gender
380035,59a5c761-f825-e311-a975-4040184c1c1a,2013,0.0,Standard,1.0,0.0,female
382378,1fbd9c37-6a79-e811-80ea-0050569e2a4f,2018,0.0,Standard,1.0,0.0,female
380017,799de6a1-130e-e511-adf0-26d4160798d6,2015,0.0,Standard,1.0,0.0,0
380641,d34bb1d1-11cd-e111-941f-00259073dc22,2010,0.0,Standard,1.0,0.0,female
380140,5d38d021-c2ef-e211-a0e0-4040184c1c1a,2013,0.0,Standard,1.0,0.0,female


In [20]:
# create a new dataframe
new_df = pd.merge(df_non_only, df_std_only, how='left', left_on=['ContactId'], right_on = ['ContactId'])

In [21]:
new_df.shape

(115554, 14)

In [22]:
new_df = new_df.fillna(0)

In [23]:
new_df.sample(5)

,ContactId,Year_x,PledgeTotal_x,VolType_x,VolunteerActivityCnt_x,BirthYear_x,Gender_x,min_year,Year_y,PledgeTotal_y,VolType_y,VolunteerActivityCnt_y,BirthYear_y,Gender_y
42371,285bb7d0-11cd-e111-941f-00259073dc22,2011,100.0,0,0.0,0.0,male,2011,0.0,0.0,0,0.0,0.0,0
58153,133edbce-11cd-e111-941f-00259073dc22,2007,52.0,0,0.0,1957.0,female,2007,0.0,0.0,0,0.0,0.0,0
112990,4c58b89b-e759-e811-80e1-0050569e2a4f,2017,50.0,0,0.0,0.0,0,2017,0.0,0.0,0,0.0,0.0,0
111606,740e7681-ac89-e611-a7f7-26d4160798d6,2016,240.0,0,0.0,0.0,male,2016,0.0,0.0,0,0.0,0.0,0
10445,98bb99cf-11cd-e111-941f-00259073dc22,2008,26.0,0,0.0,1976.0,male,2008,0.0,0.0,0,0.0,0.0,0


In [24]:
new_df = new_df.drop(new_df[new_df.VolType_y != 'Standard'].index)

In [25]:
new_df.shape

(2981, 14)

In [26]:
new_df.sample(5)

,ContactId,Year_x,PledgeTotal_x,VolType_x,VolunteerActivityCnt_x,BirthYear_x,Gender_x,min_year,Year_y,PledgeTotal_y,VolType_y,VolunteerActivityCnt_y,BirthYear_y,Gender_y
43298,bb51b1d1-11cd-e111-941f-00259073dc22,2011,30.00,0,0.0,1990.0,male,2011,2014.0,60.0,Standard,1.0,1990.0,male
98814,9350b1d1-11cd-e111-941f-00259073dc22,2010,50.00,0,0.0,0.0,female,2010,2011.0,0.0,Standard,1.0,0.0,female
86901,33e774ce-11cd-e111-941f-00259073dc22,2007,80.00,0,0.0,1973.0,male,2007,2016.0,85.0,Standard,1.0,1973.0,male
76238,7bcfa9d5-d13c-e311-b580-4040184c1c1a,2013,235.82,0,0.0,1977.0,female,2013,2015.0,0.0,Standard,1.0,1977.0,female
18219,1a23f478-386a-e211-a0e0-4040184c1c1a,2013,96.00,0,0.0,1988.0,female,2013,2012.0,96.0,Standard,1.0,1988.0,female


In [27]:
new_df_final = new_df[new_df.Year_x <= new_df.Year_y]

In [28]:
new_df_final.shape

(2504, 14)

In [29]:
new_df_final.sample(5)

,ContactId,Year_x,PledgeTotal_x,VolType_x,VolunteerActivityCnt_x,BirthYear_x,Gender_x,min_year,Year_y,PledgeTotal_y,VolType_y,VolunteerActivityCnt_y,BirthYear_y,Gender_y
87443,d3f674ce-11cd-e111-941f-00259073dc22,2007,2190.5,0,0.0,1950.0,male,2007,2016.0,10000.00,Standard,1.0,1950.0,male
63737,795cdbce-11cd-e111-941f-00259073dc22,2008,130.0,0,0.0,1969.0,female,2008,2013.0,279.76,Standard,1.0,1969.0,female
8378,ca8555a8-2382-e411-9b7f-26d4160798d6,2008,52.0,0,0.0,1958.0,female,2008,2013.0,25.00,Standard,1.0,1958.0,female
69393,ebcd62d1-11cd-e111-941f-00259073dc22,2009,35.0,0,0.0,1976.0,female,2009,2014.0,100.00,Standard,1.0,1976.0,female
105731,8e84425a-5c2d-e411-ba26-26d4160798d6,2014,264.0,0,0.0,1985.0,female,2014,2018.0,1020.07,Standard,1.0,1985.0,female


In [30]:
2504/115554

0.021669522474341

# Conclusion:

So, about **2.2%** of the non-volunteers become volunteers.  

One caveat: the case where a non-volunteer donates and then volunteers in the same year is **excluded** because there is no way to know which event happened first.  In other words the supporter could have volunteered first and then donated later in the same year.  This would not make much of a difference in the final percentage.  

A second caveat:  The supporter could eventually become a volunteer later in 2019 or in a later year.